In [8]:
# distance function
from math import sin, asin, cos, radians, fabs, sqrt
 
earth_radius=6371           # earth's average radius is 6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def get_distance_hav(lat0, lng0, lat1, lng1):
    "用haversine公式计算球面两点间的距离。"
    # The distance between two points on the sphere is calculated by haverne formula.
    # Conversion the longitude and latitude to radian
    lat0 = radians(lat0)
    lat1 = radians(lat1)
    lng0 = radians(lng0)
    lng1 = radians(lng1)
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    distance = 2 * earth_radius * asin(sqrt(h))
 
    return distance

In [3]:
import pandas as pd
import numpy as np

house = pd.read_excel('clean_house_data.xlsx')
house.columns = ['house_id','house_name','LNG','LAT','price','streetAddress','postcode','house_type']
house.drop('house_id',axis = 1, inplace = True)

In [4]:
# calculate the nearest 3 cinemas for each house
cinema = pd.read_excel('cinema_clean.xls')
cinema['index'] = [i for i in range(len(cinema))]

nearest3cinemas = []
for house_row in house.iterrows():
    nearest_index = [0,0,0]
    first_nearest = 999999
    second_nearest = 999999
    third_nearest = 999999
    for cinema_row in cinema.iterrows():
        dis = get_distance_hav(float(house_row[1][1]),float(house_row[1][2]),float(cinema_row[1][0]),float(cinema_row[1][1]))
        if dis < first_nearest: 
            third_nearest = second_nearest
            second_nearest = first_nearest
            first_nearest = dis
            nearest_index[2] = nearest_index[1]
            nearest_index[1] = nearest_index[0]
            nearest_index[0] = cinema_row[1][7]#index column num
        if dis > first_nearest and dis < second_nearest: 
            third_nearest = second_nearest
            second_nearest = dis
            nearest_index[2] = nearest_index[1]
            nearest_index[1] = cinema_row[1][7]#index column num
        if dis > second_nearest and dis < third_nearest: 
            third_nearest = dis
            nearest_index[2] = cinema_row[1][7]#index column num
    nearest3cinemas.append(nearest_index)

house['nearest_3cinemas_index'] = nearest3cinemas

In [5]:
# calculate the nearest 5 restaurants,restaurants number within 500m and the most common cuisine within 500m for each house
from collections import Counter

rest = pd.read_csv("Restaurant_clean.csv",header=0)
rest['index'] = [i for i in range(len(rest))]

common_cuisines = []
num_rests_within_500m = []
nearest5rests = []
for house_row in house.iterrows():
    cuisines = []# store each row's cuisines
    nearest_index = [0,0,0,0,0]
    first_nearest = 999999
    second_nearest = 999999
    third_nearest = 999999
    fourth_nearest = 999999
    fifth_nearest = 999999
    for rest_row in rest.iterrows():
        dis = get_distance_hav(float(house_row[1][1]),float(house_row[1][2]),float(rest_row[1][0]),float(rest_row[1][1]))*1000 
        if dis <= 500:
            split_cuisines = (rest_row[1][3]).split(';')
            for sc in split_cuisines:
                cuisines.append(sc)
                
        # calculate the nearest 5 restaurants  
        if dis < first_nearest: 
            fifth_nearest = fourth_nearest
            fourth_nearest = third_nearest
            third_nearest = second_nearest
            second_nearest = first_nearest
            first_nearest = dis
            nearest_index[4] = nearest_index[3]
            nearest_index[3] = nearest_index[2]
            nearest_index[2] = nearest_index[1]
            nearest_index[1] = nearest_index[0]
            nearest_index[0] = rest_row[1][6]#index column num
        if dis > first_nearest and dis < second_nearest:             
            fifth_nearest = fourth_nearest
            fourth_nearest = third_nearest
            third_nearest = second_nearest
            second_nearest = dis
            nearest_index[4] = nearest_index[3]
            nearest_index[3] = nearest_index[2]
            nearest_index[2] = nearest_index[1]
            nearest_index[1] = rest_row[1][6]#index column num
        if dis > second_nearest and dis < third_nearest: 
            fifth_nearest = fourth_nearest
            fourth_nearest = third_nearest
            third_nearest = dis
            nearest_index[4] = nearest_index[3]
            nearest_index[3] = nearest_index[2]
            nearest_index[2] = rest_row[1][6]#index column num
        if dis > second_nearest and dis < third_nearest: 
            fifth_nearest = fourth_nearest
            fourth_nearest = dis
            nearest_index[4] = nearest_index[3]
            nearest_index[3] = rest_row[1][6]#index column num
        if dis > second_nearest and dis < third_nearest: 
            fifth_nearest = dis
            nearest_index[4] = rest_row[1][6]#index column num
    # put them together       
    common_cuisines.append(Counter(cuisines).most_common(1))
    num_rests_within_500m.append(len(cuisines))
    nearest5rests.append(nearest_index)
    
#couner我用不太来,所以写了两个循环,一个循环总是报错
most_common_cuisine = []
for i in common_cuisines:
    if (len(i) == 0): most_common_cuisine.append("")
    else: most_common_cuisine.append(i[0][0])
#add columns
house['most_common_cuisine'] = most_common_cuisine
house['num_rests_within_500m'] = num_rests_within_500m
house['nearest_5rests_index'] = nearest5rests


#请无视下面报错

NameError: name 'num_rest_within_500m' is not defined

In [7]:
house

,house_name,LNG,LAT,price,streetAddress,postcode,house_type,nearest_3cinemas_index,most_common_cuisine,num_rests_within_500m,nearest_5rests_index
0,Beautiful 3 Bedroom Brooklyn Apt **New Kitchen...,-73.907347,40.692549,"$2,200",Eldert,11237,Apartment,"[74, 6, 62]",chinese,79,"[1453, 427, 511, 612, 442]"
1,2 Bedroom Apt in Central Location** New Full K...,-73.911805,40.692785,"$2,000",Wilson,11237,Apartment,"[74, 6, 62]",mexican,76,"[282, 513, 34, 192, 274]"
2,NO FEE 1 BR W/D IN UNIT KENSINGTON APT,-73.971044,40.633721,"$1,895",NaN,11230,Apartment,"[24, 60, 58]",new_american,1,"[1334, 8, 7, 907, 357]"
3,cNice 2 Bedroom Apt** Great Deal** Perfect Loc...,-73.860805,40.684741,"$2,150",Atlantic,11416,Apartment,"[6, 74, 62]",,0,"[1134, 771, 724, 271, 140]"
4,NO FEE 1BR APARTMENT NEAR THE Q TRAIN,-73.961334,40.622676,"$1,700",NaN,11230,Apartment,"[24, 60, 58]",pizza,2,"[357, 1279, 1334, 1274, 817]"
...,...,...,...,...,...,...,...,...,...,...,...
974,Charming Sun Swept EV 2 Bed Oasis___Amazing Lo...,-73.985753,40.726988,"$2,795",East 7th St.,10003,Apartment,"[77, 101, 103]",american,109,"[971, 1362, 1241, 818, 245]"
975,NO FEE! Large 2.5 bed 1.5 bath + W/D in Prime ...,-73.951566,40.673476,"$2,995",Park place,11216,Apartment,"[74, 112, 1]",caribbean,7,"[1293, 609, 857, 57, 485]"
976,Penthouse LES True 1 Bed Corner Stunner___Roof...,-73.992495,40.713867,"$2,195",East Broadway,10090,Apartment,"[91, 31, 15]",chinese,70,"[744, 687, 360, 92, 472]"
977,"4bed, renovated, 2 bath , roof deck. Washer/dr...",-73.937100,40.801100,"$3,690",Park Ave,10035,Apartment,"[6, 74, 62]",mexican,8,"[1170, 1065, 1281, 1072, 541]"
